# Welcome to Day 5! 

## BLAST-ing against the NCBI database

### Section 1: BLAST with genes

### Section 2: BLAST with fastq reads

### Section 3: Where to go from here

---

## Session summary

For our last day we are going to do something every biologist loves to do, BLAST. First we will learn how to BLAST against the NCBI database using a single gene of interest. Afterwards, we will learn how to BLAST reads from a fastq file against the NCBI database to see what organism they come from.

---


---
## For Google colab users only

Run the following commands

In [ ]:
pip install Biopython

In [ ]:
!wget https://raw.githubusercontent.com/agmcfarland/biopython_workshop/master/single_seq.fasta
!wget https://raw.githubusercontent.com/agmcfarland/biopython_workshop/master/short_reads.fastq
!wget https://raw.githubusercontent.com/agmcfarland/biopython_workshop/master/blast_output.xml
!wget https://raw.githubusercontent.com/agmcfarland/biopython_workshop/master/short_read_blast_results.xml

---

# Section 1: Section 1: BLAST with genes

In [ ]:
# import all our previously used modules

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# import Biopython modules for BLAST
from Bio.Blast.NCBIWWW import qblast
from Bio.Blast import NCBIXML

# import pandas to save our list of lists as dataframes
import pandas as pd

Let's say we are interested in finding homologs of ermA from Staphylococcus aureus in the NCBI non-redundant database. 

We have a fasta file called `single_seq.fasta`, which contains only one ermA sequence. We use `SeqIO.read()` (instead of `SeqIO.parse()`) because there is only one sequence.

In [ ]:
record = SeqIO.read('single_seq.fasta','fasta')
record

Let's use `qblast()` to search for homologs of our sequence. [View full qblast options here](https://biopython.org/docs/1.75/api/Bio.Blast.NCBIWWW.html)

In [ ]:
# searching for homologs of record.seq in the non-redudundant (nr) database using BLASTn. BLAST will only return the top five matches (hits).
# these are only just some of the options we can specify in our search. The first three are mandatory, however.
blast_results = qblast(program='blastn', database='nr', sequence=record.seq, hitlist_size=5) 

# storing the results of the blast search in a file called blast_output.xml 
output_file = open('blast_output.xml', 'w') # the 'w' indicates we are writing to this file.
output_file.write(blast_results.read()) # write blast_results to output_file
output_file.close() # closing the file we wrote to
blast_results.close() # closing the blast result

Each blast 'hit', (i.e. a sequence in the database that matched our sequence of interest) is an item in the list `blast_record.alignments`.

Let's check that five hits matching our sequence of interest were found 

In [ ]:
blast_result_handle = open('blast_output.xml') # open the blast output file
blast_record = NCBIXML.read(blast_result_handle) # read it into memory

count = 0
for alignment in blast_record.alignments:
    count = count + 1

count

We can print the name of each sequence that matched our search sequence.

In [ ]:
# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('blast_output.xml')
blast_record = NCBIXML.read(blast_result_handle)

for alignment in blast_record.alignments:
    print(alignment.title)

We can access additional information stored in each alignment with a second for loop. These are stored in a list called `alignment.hsps` and provide all the important information about the alignment.

Information found in `alignment.hsps` includes the length of the alignment, which bases did and did not align, and even the alignment itself! 

In [ ]:
# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('blast_output.xml')
blast_record = NCBIXML.read(blast_result_handle)

for alignment in blast_record.alignments:
    print(alignment.title.split('>')[0])
    for hsp in alignment.hsps:
        print('score: ', hsp.score)
        print('expected value: ', hsp.expect)
        print('number of exact matches: ', hsp.identities)
        print('number of aligned letters: ', hsp.align_length)
        print('First 100 characters aligned: \n') # for visual clarity only showing first 100 characters
        print(hsp.query[:100])
        print(hsp.match[:100])
        print(hsp.sbjct[:100])
        print('\n')
        break

In practice, we may just want to extract certain bits of information from the BLAST search

For example, we can extract a list of each genome we found a hit in

In [ ]:
# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('blast_output.xml')
blast_record = NCBIXML.read(blast_result_handle)

store_data = []

for alignment in blast_record.alignments:
    hit_title = alignment.title.split('>')
    store_data.append(hit_title)

store_data

This list shows us that although we searched for a Staphylococcus aureus sequence, we have found a match in a plasmid contained within an Enterococcus faecium strain.

We can calculate strength in homology of each match. We do this by calculating the identity (number of identical bases between two sequences) and the query coverage (how many bases were aligned between the two sequences)

In [ ]:
# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('blast_output.xml')
blast_record = NCBIXML.read(blast_result_handle)

store_data = []

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        # make variables for easier readability
        length_sbjct = len(hsp.sbjct)
        length_query = len(hsp.query)
        num_gaps = hsp.gaps
        num_identities = hsp.identities
        
        # calculate identity
        identity = num_identities/length_query

        # calculate coverage
        coverage = (length_sbjct-num_gaps)/length_query
        
        # append to store_data
        store_data.append([alignment.title, identity, coverage])
        break

store_data

In [ ]:
df = pd.DataFrame(store_data, columns=['hit_id','identity','coverage'])
df

Suprisingly, all sequences show 81.7% identity and 100% coverage to our query sequence, including the plasmid-borne version.

This is just one utility of parsing BLAST outputs with Biopython. 

Another utility is to extract all the sequences that you have gotten through BLAST and writing them to a new file.

---

### Exercise 1a

Use `SeqIO.read()` to read in the only sequence in `single_seq.fasta`

`translate()` `record.seq` so that it has an amino acid sequence.

In [ ]:
record = SeqIO.___('___.fasta','fasta')

record.seq = record.seq.___()
record

### Exercise 1b

Specify the `qblast()` program to be `'blastp'`, with the `'nr'` database. Make the number of max number of hits equal to `4`.

Afterwards, store the blast output in the file `'day5_1b_output.xml'` 

(Run the next line of code to get the blast results if blast is taking too long)

In [ ]:
blast_results = qblast(program='___', database='___', sequence=record.seq, hitlist_size=___)

output_file = open('___.xml', 'w')
output_file.write(blast_results.read())
output_file.close()
blast_results.close()

In [ ]:
!wget https://raw.githubusercontent.com/agmcfarland/biopython_workshop/master/day5_1b_output.xml

### Exercise 1c:

Read the contents of `blast_result_handle` with `NCBIXML.read` and assign the contents  to `blast_record`.

Print the `alignment.title` in each `blast_record.alignment`. Is there something unexpected?

In [ ]:
# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('day5_1b_output.xml')
___ = ___.read(blast_result_handle)

for alignment in blast_record.alignments:
    print(alignment.___)
    print('\n')

### Exercise 1d

 If we look at the above output, we can see that the third alignment title has the term MULTISPECIES in it. This term denotes a sequence that is identical sequence in found in multiple species. When this happens, NCBI has its `alignment.title` as a long string with all the different species names. 
 
 But we just want to shorten the name for our purposes.

 We can see that the `alignment.title` with the MULTISPECIES term has each different species separated by a `>`. We can extract the contents of `alignment.title` using  `split()` and keep only the first species that is found.


In [ ]:
# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('day5_1b_output.xml')
___ = ___.read(blast_result_handle)

for ___ in blast_record.alignments:
    # use split() on alignment.title
    new_alignment_title = alignment.title.___('>')

    #select the first item in new_alignment_title, assign it back to new_alignment_title
    new_alignment_title = ___[0]

    # print the new title
    print(new_alignment_title)
    print('\n') # for clarity

### Exercise 1f

Now we want to extract the new alignment title and the sequence for each hit so that we can write them to a `fasta` file

Convert the `hsp.sbjct` sequence into a `Seq()` object. 

Afterwards, make a `SeqRecord` object from the extracted sequence and the extracted title.

Store the `new_alignment_title` and the `hsp.sbjct` in the list `data_store` 

Read the code comments for clues!

In [ ]:
from Bio.SeqRecord import SeqRecord

data_store = []

# blast_record has to be read with NCBIXML each time you call the for loop
blast_result_handle = open('day5_1b_output.xml')
___ = ___.read(blast_result_handle)

for alignment in blast_record.alignments:
    # use split()
    new_alignment_title = alignment.title.___('>')

    #select the first item in new_alignment_title, assign it back to new_alignment_title
    new_alignment_title = ___[0]

    # assign hsp.sbjct to sbjct_sequence
    for hsp in alignment.hsps:
        sbjct_sequence = hsp.___
        break
    
    # Make a Seq object out of sbjct_sequence, Assign it to extracted_sequence 
    extracted_sequence = Seq(___)

    # make a SeqRecord object called extracted_record. The first item will be the sequence, id is the new alignment title
    extracted_record = SeqRecord(___, id = ___, description = '')

    # store the SeqRecord in a list
    data_store.append(extracted_record)
data_store

### Exercise 1g

Use `SeqIO.write()` to write the contents of `data_store` to a file called `'day5_1g.fasta'`. Specify the output file to be of type `'fasta'`



In [ ]:
SeqIO.___(___,'day5_1g.fasta','___')

# Section 2: BLAST with fastq reads

One quick method for checking whether your reads actually belong to the organism you are studying is to BLAST some of the reads against the NCBI database.

Unlike in our above code where we were supplied `qlbast()` with just **one** sequence to BLAST, here we want to BLAST several sequences, and have each give its own result.

Before we can do that, though, we need to convert our reads from `fastq` format to `fasta` format.


We will start by reading in the fastq file `short_reads.fastq`, which contains reads from whole-genome sequencing of a Pseudomonas monteilii strain. 

We will extract their `record.id` and `record.seq` and store them in a list.

In [ ]:
# empty list to store modified sequence records
extracted_seqs = []

for record in SeqIO.parse('short_reads.fastq', 'fastq'):
    # get the id and sequence assigned to two new variables
    read_id = record.id
    dna_seq = record.seq
    
    # format them into a SeqRecord. keep description empty
    new_record = SeqRecord(dna_seq, id = read_id, description = '')

    # append to extracted_seqs
    extracted_seqs.append(new_record)

    print(new_record)


SeqIO.write(extracted_seqs, 'short_fasta_reads.fasta','fasta')

Now that we have our reads in fasta format, we can supply the whole all the sequences at once to `qblast()`.

We are setting our `hitlist_size` to 1. This is because we want to keep our results simple: one sequence homology BLAST hit per read. We will also ask that it

In [ ]:
# open short read fasta sequences using open() 
read_seqs = open('short_fasta_reads.fasta')
# place the contents of read_seq into memory with read()
read_seqs_memory = read_seqs.read()
# supply qblast parameters
blast_results = qblast(program='blastn', database='nr', sequence=read_seqs_memory, hitlist_size=1)

# write the contents of the blast result to a file called 'short_read_blast_results.xml'
output_file = open('short_read_blast_results.xml', 'w')
output_file.write(blast_results.read())

# close the output file
output_file.close()
# close the blast_results file
blast_results.close()
# close the fasta file
read_seqs.close()

In [ ]:
blast_result_handle = open('short_read_blast_results.xml') # open the blast output file
blast_records = NCBIXML.parse(blast_result_handle) # use parse to read a blast result with more than one query

for blast_record in blast_records:
    print('Read ID: ', blast_record.query)
    for alignment in blast_record.alignments:
        print('Hit title: ,', alignment.title)
    print('\n')

We expected to see Pseudomonas only matches. However, the second read matches Cosmia trapezina. In the next exercise let's re-run our blast search and allow for 3 matches per query.

---

### Exercise 2a

Submit the sequences in `'short_fasta_reads.fasta'` to `qblast()`.

Use program `'blastn'`, the `'nr'` database, sequence `read_seqs_memory`, and set `hitlist_size` to `3`

Write the contents to a file called `'day5_2a.xml'`

(Run the next line of code to get the blast results if blast is taking too long)


In [ ]:
# open short read fasta sequences using open() 
read_seqs = open('short_fasta_reads.fasta')
# place the contents of read_seq into memory with read()
read_seqs_memory = read_seqs.read()
# supply qblast parameters
blast_results = qblast(program='___', database='___', sequence=___, hitlist_size=___)

# write the contents of the blast_results to a file called 'day5_2a.xml'
output_file = open('day5_2a.xml', 'w')
output_file.write(___.read())

# close the output file
output_file.___()
# close the blast_results file
____.close()
# close the fasta file
read_seqs.close()

In [ ]:
!wget https://raw.githubusercontent.com/agmcfarland/biopython_workshop/master/day5_2a_output.xml

### Exercise 2b

For each `blast_record`, print the `blast_record.query` in the blast result file, print the `alignment.title` of the hit.

In [ ]:
blast_result_handle = open('day5_2a.xml') # open the blast output file
blast_records = NCBIXML.parse(blast_result_handle) # use parse to read a blast result with more than one query

for ___ in blast_records:
    print('Read ID: ', ___.___)
    for alignment in blast_record.alignments:
        print('Hit title:', alignment.___)
    print('\n')

### Exercise 2c

Our results indicate that most reads return Pseudomonas matches, which is expected. However, read 2 had only two total hits found (and had up to 10 hits available to return). This suggests that read 2 is not contamination, but just a bad read. We can determine whether this is the case by examining the strength of the homology of our reads. If the homology strength for read 2 is high, then it is a contaminant read. If it is low, it is likely that it is simply a low-quality read.

For each query in the blast result file, print the `alignment.title` of the hit and the `hsp.expect` (e-value) within `alignment.hsps` to get the strength of homology



In [ ]:
blast_result_handle = open('day5_2a.xml') # open the blast output file
blast_records = ___.___(blast_result_handle) # use NCBIXML.parse to read a blast result with more than one query

for ___ in blast_records:
    print('Read ID: ', blast_record.___)
    for alignment in blast_record.alignments:
        print('Hit title:', alignment.title)
        for ___ in alignment.hsps:
            print('Expected value: ', hsp.expect)
    print('\n')

### Exercise 2d

From our results we can see two things. Most hits have a very [low e-value](http://resources.qiagenbioinformatics.com/manuals/clcgenomicsworkbench/650/_E_value.html#:~:text=The%20default%20threshold%20for%20the,%3C%2010e%2D100%20Identical%20sequences) (less than 1 ). Hits to read 2 have e-values of 4.

Create a variable called `e_value_threshold` with a value of `0.0001`

Let's filter our results so that only `hsp.expect` (evalues) of a certain value are printed.

In [ ]:
blast_result_handle = open('day5_2a.xml') # open the blast output file
blast_records = NCBIXML.parse(blast_result_handle) # use parse to read a blast result with more than one query

e_value_threshold = ___

for blast_record in blast_records:
    print('Read ID: ', blast_record.query)
    for alignment in blast_record.alignments:
        print('Hit title:', alignment.title)
        for ___ in alignment.hsps:
            if hsp.___ <= e_value_threshold:
                print('Expected value: ', hsp.expect)
            else:
                print('Expected value: did not pass')
            break
    print('\n')

### Exercise 2e

We will now store our results to a list called `store_results` using `append()`

In [ ]:
blast_result_handle = open('day5_2a.xml') # open the blast output file
blast_records = NCBIXML.parse(blast_result_handle) # use parse to read a blast result with more than one query

store_results = ___

___ = 0.0001

for blast_record in blast_records:
    # get the read_id
    read_id = blast_record.query

    for alignment in blast_record.alignments:
        # get the name of the hit
        hit_title = alignment.title

        for hsp in alignment.hsps:
            # get the evalue from hsp.expect
            evalue = hsp.___

            # get a qualitative result for the evalue threshold
            if hsp.expect <= e_value_threshold:
                evalue_pass = 1
            else:
                evalue_pass = 0
            
            # append to list
            store_results.___([read_id, hit_title, evalue, evalue_pass])


store_results

# Section 3: Where to go from here

This workshop covered a lot of ways to use Biopython, but is not exhaustive. 

If you are interested in further developing your Python and Biopython skills and toolkits, consider the following resources:

[Northwestern's Research Computing Workshops: Learn additional programming concepts, languages, applications](https://www.it.northwestern.edu/service-catalog/research/training/workshops.html)

[Biopython cookbook: the official Biopython guide to using all the different Biopython modules and subpackages](https://biopython.org/wiki/Category%3ACookbook)

[Rosalind: Lots of free bioinformatic/computational biology challenges, ranging from easy to tough. You can use just base python and/or biopython to answer the questions.](http://rosalind.info/problems/locations/)

[Biostar forums: Lots of good discussion about programming and biology.](https://www.biostars.org/t/Forum/)

[Bioinformatics stack exchange: Discussion, questions, and answers about bioinformatics and computational biology](https://bioinformatics.stackexchange.com/)

[Learning how to use command-line blast against a locally stored database](https://angus.readthedocs.io/en/2019/running-command-line-blast.html)

[Use Northwestern's Research Data Services: if you are working on programming scripts (including those using Biopython) for your research and would like to recieve a consultation. They also provided consultations for any data science-related projects/scripts/problems](https://www.it.northwestern.edu/research/consultation/data-services.html)




That's it for Day 5! Thanks for attending this Biopython workshop!